In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the fourier_learning_ibm package to the path
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import networkx as nx
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    SamplerV2 as Sampler,
    Batch,
)
from heisenberg import (
    HeisenbergModel,
    get_n_steps,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
)
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.quantum_info.operators import Operator
from qiskit.primitives import BackendSamplerV2
from qiskit.quantum_info import Statevector, state_fidelity
import mthree
import os
import time
from datetime import datetime, timezone
import json
import pickle
from typing import List, Tuple, Dict

In [ ]:
n_qubits = 10

# total time to be simulated
# type is list so that it can be saved to json
total_time = 15
times = np.linspace(0, total_time, 30).tolist()

rng = np.random.default_rng(42)

## Create graph (Demo)

In [ ]:
graph_type = "line"
G = get_graph(n_qubits, rng, graph_type)

positions = get_positions(n_qubits, graph_type)

# エッジラベルを作成
edge_J_labels = {edge: f"{G.edges[edge]['J']:.2g}" for edge in G.edges}
# edge_interaction_order_labels = {
#     edge: f"{G.edges[edge]['interaction_order']}" for edge in G.edges
# }
edge_cnot_order_labels = {edge: f"{G.edges[edge]['cnot']['order']}" for edge in G.edges}

# グラフを描画
plt.figure(figsize=(10, 3))
nx.draw(
    G,
    pos=positions,
    with_labels=True,
    node_color=["red" if G.nodes[node]["hadamard"] else "black" for node in G.nodes],
    node_size=300,
    edge_color="gray",
    font_color="white",
    font_size=10,
)

# エッジの重みを描画
nx.draw_networkx_edge_labels(
    G,
    pos=positions,
    edge_labels=edge_J_labels,
    font_size=10,
    font_color="black",
    label_pos=0.6,
    verticalalignment="top",
)

# エッジの 'cnot' 'order' 属性を描画
nx.draw_networkx_edge_labels(
    G,
    pos=positions,
    edge_labels=edge_cnot_order_labels,
    font_size=10,
    font_color="green",
    label_pos=0.8,
    verticalalignment="bottom",
)

plt.show()

In [ ]:
# Option1: Use IBM Quantum backend.
# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")

# Option2: Use local AerSimulator as the backend.
backend_sim = AerSimulator()
noise_model = NoiseModel.from_backend(backend_qpu)
backend_sim_noisy = AerSimulator(noise_model=noise_model)
# Note: statevector simulator
# backend_sim_noisy = AerSimulator(method="statevector", noise_model=noise_model)

print(f"Using backend QPU: {backend_qpu}")
print(f"Using backend simulator: {backend_sim}")
print(f"Using backend noisy simulator: {backend_sim_noisy}")

In [ ]:
rng = np.random.default_rng(42)
n_steps_range = np.arange(1, 11)
n_samples = 20
graphs = [get_graph(n_qubits, rng, graph_type) for _ in range(n_samples)]

results_sim_noisy: dict[int, dict[int, list[object]]] = {
    # int: sample_id
    # dict[int, list[object]]:
    #     int: n_steps
    #     list[object]: results
}

states_exact: dict[int, list[object]] = {
    # int: sample_id
    # list[object]: states
}
states_sim_noisy: dict[int, dict[int, list[object]]] = {
    # int: sample_id
    # dict[int, list[object]]:
    #     int: n_steps
    #     list[object]: states
}

$n_{\mathrm{sample}}$ 個のハミルトニアンを exact と simulator で時間発展させて、fidelity を調べる

### Exact simulation

In [ ]:
for i in range(n_samples):
    G = graphs[i]
    heisenberg = HeisenbergModel(n_qubits, G, backend_sim_noisy)
    states_exact[i] = []

    for k in range(len(times)):
        print(f"sample: {i}, k: {k}, time: {times[k]}")
        state_exact = heisenberg.exact_simulation_state(times[k], phase=0)
        states_exact[i].append(state_exact)

### Trotter simulation

In [ ]:
# Only for CP1
%cd fourier_learning_ibm/

In [ ]:
for i in range(n_samples):
    G = graphs[i]
    heisenberg = HeisenbergModel(n_qubits, G, backend_sim_noisy)
    states_sim_noisy[i] = {
        # key (int): n_steps
        # value (list[object]): states
    }

    for n_steps in n_steps_range:
        states_sim_noisy[i][n_steps] = []

        for k in range(len(times)):
            print(f"Sample {i+1}, n_steps: {n_steps}, k: {k}, time: {times[k]}")
            circuit, exec_circuit = heisenberg.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
            # Remove final measurements
            exec_circuit.remove_final_measurements()
            # Save the statevector at the end of the circuit
            exec_circuit.save_statevector()

            result_noisy = backend_sim_noisy.run(exec_circuit).result()
            states_sim_noisy[i][n_steps].append(result_noisy.get_statevector())

# Save the results to binary and json
CURRENT_TIME = (
    datetime.now(timezone.utc).isoformat(timespec="minutes").replace("+00:00", "Z")
)
path = f"./test_trotter_step/{CURRENT_TIME}"
os.makedirs(path, exist_ok=True)
print(f"CURRENT_TIME: {CURRENT_TIME}")
print(f"Saving data to {path}")

# Save the graphs as a binary file
with open(f"{path}/graphs.pkl", "wb") as f:
    pickle.dump(
        {
            "graphs": graphs,
        },
        f,
    )

fidelities: dict[int, dict[int, list[float]]] = {
    # int: sample_id
    # dict[int, list[float]]:
    #     int: n_steps
    #     list[float]: fidelities
}

for i in range(n_samples):
    print(f"Sample {i+1}/{n_samples}")

    fidelities[i] = {
        # key (int): n_steps
        # value (list[float]): fidelities
    }

    for n_steps in n_steps_range:
        print(f"n_steps: {n_steps}")
        fidelities[i][int(n_steps)] = []

        for k in range(len(times)):
            print(f"k: {k}, time: {times[k]}")
            state_exact = states_exact[i][k]
            state_sim_noisy = states_sim_noisy[i][n_steps][k]

            fidelity = state_fidelity(state_exact, state_sim_noisy)
            fidelities[i][int(n_steps)].append(fidelity)


# n_steps ごとに、fidelity の平均と標準偏差を計算
fidelities_avg: dict[int, list[float]] = {
    # int: n_steps
    # list[float]: fidelity_avg
}
fidelities_std: dict[int, list[float]] = {
    # int: n_steps
    # list[float]: fidelity_std
}

for n_steps in n_steps_range:
    fidelities_avg[int(n_steps)] = []
    fidelities_std[int(n_steps)] = []

    for k in range(len(times)):
        fidelities_avg[int(n_steps)].append(
            np.mean([fidelities[i][int(n_steps)][k] for i in range(n_samples)])
        )
        fidelities_std[int(n_steps)].append(
            np.std([fidelities[i][int(n_steps)][k] for i in range(n_samples)])
        )


# Save the data as a JSON file
with open(f"{path}/data.json", "w") as f:
    json.dump(
        {
            "n_qubits": n_qubits,
            "backend_qpu_name": backend_qpu.name,
            "times": times,
            "len(times)": len(times),
            "n_steps_range": n_steps_range.tolist(),
            "n_samples": n_samples,
            "fidelities": fidelities,
            "fidelities_avg": fidelities_avg,
            "fidelities_std": fidelities_std,
        },
        f,
        indent=4,
    )

In [ ]:
# Load the data from the JSON file
with open(f"{path}/data.json", "r") as f:
    data = json.load(f)

n_qubits = data["n_qubits"]
backend_qpu_name = data["backend_qpu_name"]
times = data["times"]
n_steps_range = np.array(data["n_steps_range"])
n_samples = data["n_samples"]
fidelities = data["fidelities"]
fidelities_avg = data["fidelities_avg"]
fidelities_std = data["fidelities_std"]

In [ ]:
# Plot the results
plt.figure(figsize=(10, 5))
for n_steps in n_steps_range:
    plt.errorbar(
        times,
        fidelities_avg[n_steps],
        yerr=fidelities_std[n_steps],
        label=f"{n_steps} steps",
        marker="^",
    )
plt.title(
    f"Fidelity between exact and Trotter simulation on {n_qubits} qubit, on average over {n_samples} samples"
)
plt.xlabel("Time")
plt.ylabel(f"Fidelity")
plt.ylim(0, 1.05)
plt.legend(bbox_to_anchor=(0, -0.1), loc="upper left")
plt.show()